# 0 Preparation

In [2]:
import re
import spacy
import zipfile
import json
nlp = spacy.load("en_core_web_sm")
!unzip -j /content/reports.zip

Archive:  /content/reports.zip
 extracting: icsma-20-184-01.txt     
 extracting: icsa-20-184-01.txt      
 extracting: icsa-20-184-02.txt      
 extracting: icsa-20-182-01.txt      
 extracting: icsa-20-182-02.txt      
 extracting: icsma-20-177-01.txt     
 extracting: icsa-20-177-01.txt      
 extracting: icsa-20-177-02.txt      
 extracting: icsa-20-177-03.txt      
 extracting: icsa-20-175-01.txt      
 extracting: icsa-20-175-02.txt      
 extracting: icsa-20-175-03.txt      
 extracting: icsma-20-170-01.txt     
 extracting: icsma-20-170-02.txt     
 extracting: icsma-20-170-03.txt     
 extracting: icsma-20-170-04.txt     
 extracting: icsma-20-170-06.txt     
 extracting: icsma-20-170-05.txt     
 extracting: icsa-20-170-02.txt      
 extracting: icsa-20-170-03.txt      
 extracting: icsa-20-170-04.txt      
 extracting: icsa-20-170-05.txt      
 extracting: icsa-20-168-01.txt      
 extracting: icsa-20-170-01.txt      
 extracting: icsa-20-161-02.txt      
 extracting: icsma-

In [3]:
import pandas as pd
import numpy as np
import random
from bs4 import BeautifulSoup
import nltk
from nltk import word_tokenize
from nltk import tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
# Read the previous examples of vendors
df_vendor = pd.read_csv("vendors.csv",header=None)
vendor_list = list(df_vendor[0].dropna())

### 0.0 Text Processing

In [5]:
# A functin to read all the lines from pure text advisory report
def read_file(file_name):
  file_content = []
  with open(file_name, "r") as f:
    read_line = f.readlines()
  for i in range(len(read_line)):
    if read_line[i].rstrip()=='':
      continue
    else:
      file_content.append(read_line[i].rstrip())
  return file_content

In [6]:
# A function to judge whether the only word in the list
def text_equal(line, lst):
  st = re.search('[A-Za-z]',line)
  s = 0
  if st is not None:
    s = st.start()
  line = line[s:]
  for i in lst:
    if line.upper() == i:
      return 1
  return 0

In [7]:
# A function to see whether the line is a divider for vulneralbility section
def check_line(j):
  for i in j.split():
    if i == 'of':
      continue
    elif i == 'for':
      continue
    elif '---------' in i:
      return 0
    elif i == i.title():
      continue
    elif i == i.upper():
      continue
    elif '-' in i:
      continue
    else:
      return 0

  if j[-1] == '.':
    return 0
  elif len(j) <= 4:
    return 0
  elif len(j.split())<=4:
    return 1

  return 1

In [8]:
# A function to check whether a string contains word(s) from a list
def find_sub(string, lst, find_all = 0):
  # all the keywords should be in the list
  if find_all:
    count=0
    for i in lst:
      if i in string:
        count+=1
    if count==len(lst):
      return 1
    else:
      return 0
  # only one of the keywords in the string would be enough
  else:
    for i in lst:
      if i in string:
        return 1
    return 0

In [9]:
# A function to locate the position of certain regex
def find_key(line,kwds):
  m_min = len(line)
  for i in kwds:
    m = re.search(r'{}'.format(i), line)
    if m is not None:
      m = m.start()
      # get the smallest position of keywords
      if m < m_min:
        m_min = m
  return m_min

In [10]:
# A function to strip noise from a string
def polish(text):
  text = text.strip()
  text = text.strip('.')
  text = text.strip(',')
  text = text.strip('•')
  text = text.strip(':')
  text = text.strip(';')
  text = text.strip('–')
  text = text.strip('-')
  text = text.strip('/')
  text = re.sub('( |,)?and$','',text)
  text = re.sub('( |,)?with ?$','',text)
  text = re.sub(' product versions$','',text)
  for i in ['\) and',', and']:
    if re.search(i,text) is not None:
      if re.search(i,text).end() == len(text):
        t = re.search(i,text).start()
        text = text[:t]
  text = text.rstrip('(')
  text = text.rstrip('[')
  text = text.lstrip(')')
  text = text.lstrip(']')
  text = re.sub(',,',',',text)
  if re.search('\(',text) is not None:
    if not re.search('\(',text).start():
      text = text.lstrip('(')
      text = text.rstrip(')')
  if text[:2] == 'll':
    text = 'a'+text
  return text

### 0.1 Text Reading


#### 0.1.1 Utilities

In [11]:
# utilities for text_reading
affect_prod_end = ['impact','overview','vulnerability','cwe-538','background']
affect_prod_skip = ['Begin Update','End Update']
feed_nxt = ['(incl.','incl.','Incl.','Incl','LOGO!','Cat.']
feed_nxt_incl = ['(incl.','incl.']

#### 0.1.2 Functions

In [12]:
# A function to collect lines in the affected products section
def collect_lines(i):
  
  content = []
  tok_content = []
  add_line_flag = 0
  af_flag = 0

  file_content = read_file(i)
  noise = '\. \. \. -'

  for j in file_content:
    if '--------' in j:
      continue
    j = re.sub(noise,'',j)
    # stop if the ending line occurs 
    if j.split()[-1].lower() in affect_prod_end and add_line_flag:
      if j[0]!='·':
        add_line_flag = 0  
    # read and store the line if the first divider has been identified            
    if add_line_flag and not find_sub(j, affect_prod_skip):
      tok_sent = tokenize.sent_tokenize(j)
      tok_content = tok_content + tok_sent
    # initiate reading when the first line occurs
    if 'AFFECTED PRODUCT'.upper() in re.sub(r'\xa0',' ',j) and not af_flag:
      add_line_flag = 1
      af_flag = 1
    if 'AFFECTED PRODUCT'.title() in re.sub(r'\xa0',' ',j) and not af_flag:
      add_line_flag = 1 
      af_flag = 1
  
  # filter and complement sentences that are mistakenly split
  num = 0
  while num < len(tok_content):
    sent = tok_content[num]
    # abandon sentences with only symbols
    if re.sub('[A-Za-z0-9]','',sent) == sent:
      num+=1
      continue
    # connect tokens to rebuild split sentences
    elif re.sub('[A-Za-z]','',sent) == sent:
      up = re.sub('[A-Za-z]','',content[-1]) != content[-1]
      if num < len(tok_content)-1:
        low = re.sub('[A-Za-z]','',tok_content[num+1]) != tok_content[num+1]
      else:
        low = 1
      if up and low:
        content[-1] = content[-1] + sent
      else:
        content.append(sent)
    elif find_sub(sent.split()[-1],feed_nxt):
      num += 1
      lost_part = tok_content[num]
      while find_sub(lost_part.split()[-1],feed_nxt_incl):
        sent = sent + re.sub('·',' ',lost_part)
        num += 1
        lost_part = tok_content[num]
      sent = sent + re.sub('·',' ',lost_part)
      content.append(sent)
    elif sent.split()[-1]=='Controllers':
      num += 1
      lost_part = tok_content[num]
      while '·Series' == lost_part.split()[0]:
        temp = sent + re.sub('·',' ',lost_part)
        content.append(temp)
        num += 1
        if num == len(tok_content):
            num += 1
            break
        lost_part = tok_content[num]
      num -= 1
    elif sent.split()[-1]=='with':
      flag = 1
      num += 1
      lost_part = tok_content[num]
      while 'with' not in lost_part.split()[-1]:
        if get_div(lost_part) == 1:
          content.append(lost_part)
          num += 1
        else:
          lost_part = lost_part.split(' are included ')[0]
          temp = sent + re.sub('·',' ',lost_part)
          content.append(temp)
          num += 1
        if num == len(tok_content):
          num += 1
          break
        lost_part = tok_content[num]
      num -= 1
    else:
      content.append(sent)
    num += 1

  return [i, content]

# 1 General Information

## 1.1 Utilities

In [13]:
# utilities for getting vendor information
new = ['Rockwell','Yokogawa','3S CODESYS','Wonderware','OpenSSL',
       'Certec','Sauter','Motorola','IniNet','SchneiderWEB','St. Jude',
       'IBHsoftec','DNP3','Everest','eWON','Janitza UMG Power','Tollgrade',
       'B. Braun Medical SpaceCom','B+B SmartWorx','PACTware','Kepware',
       'Post Oak','Endress+Hauser','Pepperl+Fuchs','Fox','NTP','Cisco',
       'Triangle Research','Canary Labs Inc','Trane Tracer SC','Top Server',
       'B+B SmartWorx','Pro-Face','Cimon','BMC Medical and 3B Medical',
       'BD','RealFlex']
vul_sec = ['VULNERABILITY OVERVIEW','VUNLNERABILITY OVERVIEW',
           'VULNERABILITY CHARACTERIZATION','VULNERABILITY',
           'FILE AND DIRECTORY INFORMATION EXPOSURE CWE-538']
vul_gap = ['VULNERABILITY DETAILS','BACKGROUND','RESEARCHER','MITIGATION',
           'VUNLNERABILITY DETAILS']
vul_err = ['3.3 BACKGROUND','VULNERABILITY OVERVIEW','VULNERABILIY OVERVIEW']
# utilities for side cases
nt_gap = ['VULNERABILITY DETAILS','BACKGROUND','RESEARCHER','MITIGATION',
          'VUNLNERABILITY DETAILS']
nt_sec = ['VULNERABILITY OVERVIEW','VUNLNERABILITY OVERVIEW',
          'VULNERABILITY CHARACTERIZATION']

## 1.2 Get Vendors

In [14]:
# A function to get the vendor corresponding to the advisory
def get_vendor(i):
  vendor = ''
  for j in read_file(i):
    j = re.sub('·','',j)
    if 'vendor' in j.lower():
      if not re.search('vendor', j.lower()).start():
        vd = re.search('vendor', j.lower()).end()
        vendor = j[vd:]
        vendor = vendor.strip(':')
        vendor = vendor.strip()
  if vendor == '':
    pot_vendor = read_file(i)[1]
    for j in vendor_list:
      if re.search(j.lower(),pot_vendor.lower()) is not None:
        vendor = j
  if vendor == '':
    pot_vendor = read_file(i)[1]
    for j in new:
      if j.lower() in pot_vendor.lower():
        vendor = j
        
  return vendor

## 1.3 Get the Vulnerability Parameter

In [15]:
# A function to initiate the information of a vulnerability
def init():
  output = {}
  output['Type'] = ''
  output['CWE'] = ''
  output['CVE'] = ''
  output['CVSS'] = {}
  output['CVSS']['version'] = ''
  output['CVSS']['base score'] = ''
  output['CVSS']['temporal score'] = ''
  output['CVSS']['vector'] = ''
  output['Intrusion'] = ''
  return output

In [16]:
# A function to find the footnotes in a line
def find_ft(t,i):
  pt = '·'+t+'\.'
  for j in read_file(i):
    if re.search(r'{}'.format(pt),j) is not None:
      if not re.search(r'{}'.format(pt),j).start():
        cwe = find_cwe(i,j)
        return cwe

In [17]:
# A function to find the CWE information in a line
def find_cwe(f,i):
  if "CWE-" in i:
    v = re.search(r"CWE-[0-9]{1,}",i)
    if v is None:
      v = re.search(r"CWE- [0-9]{1,}",i)
    cwe = v.group()
  elif '[Footnote' in i:
    s = re.search('\[Footnote:',i).end()
    ft = i[s:s+2].rstrip(']')
    ft = ft.rstrip(',')
    cwe=find_ft(ft,f)
  else:
    cwe=''
  return cwe

In [18]:
# A function to find the CVSS version in a line
def find_cvss(f):
  ver = ''
  for i in read_file(f):
    if "CVSS" in i:
      if re.search('CVSS V[0-9] [0-9]\.[0-9]', i.upper()) is not None:
        ver = re.search('CVSS V[0-9] [0-9]\.[0-9]', i.upper()).group()
        ver = re.sub('CVSS','',ver).strip()
        return ver
      elif re.search('CVSS V[0-9]', i.upper()) is not None:
        ver = re.search('CVSS V[0-9]', i.upper()).group()
        ver = re.sub('CVSS','',ver).strip()
        return ver
      elif re.search('CVSSVersion [0-9].[0-9]', i) is not None:
        ver = re.search('CVSSVersion [0-9].[0-9]', i).group()
        ver = re.sub('CVSS','',ver).strip()
        return ver
      elif re.search('CVSS Version [0-9].[0-9]', i) is not None:
        ver = re.search('CVSS Version [0-9].[0-9]', i).group()
        ver = re.sub('CVSS ','',ver).strip()
        return ver
      elif re.search('CVSSV[0-9]', i.upper()) is not None:
        ver = re.search('CVSSV[0-9]', i.upper()).group()
        ver = re.sub('CVSS','',ver).strip()
        return ver
      elif re.search('CVSS:[0-9]\.[0-9]', i.upper()) is not None:
        ver = re.search('CVSS:[0-9]\.[0-9]', i.upper()).group()
        ver = re.sub('CVSS:','',ver).strip()
        return ver
      elif re.search('CVSSd Version [0-9].[0-9]', i) is not None:
        ver = re.search('CVSSd Version [0-9].[0-9]', i).group()
        ver = re.sub('CVSSd ','',ver).strip()
        return ver
  return ver

In [19]:
# A function to find vulnerability informatin in the section
def process_vul(i, vul):
  # find the CVSS vector
  if re.search('([A-Za-z]+:[A-Za-z]+/?){2,}', i) is not None:
    vec = re.search('([A-Za-z]+:[A-Za-z]+/?){2,}', i).group()
    vul['CVSS']['vector'] = vec
  # find the CVE information
  if "CVE-" in i:
    t = re.sub(' ','',i)
    vul['CVE'] = re.search(r"CVE-[0-9]{4}-[0-9]{4,}",t).group()
  # find the CWE information
  if "CWE-" in i:
    v = re.search(r"CWE-[0-9]{1,}",i)
    if v is None:
      v = re.search(r"CWE- [0-9]{1,}",i)
    vul['CWE'] = v.group()
  # find the CVSS version
  if "CVSS" in i:
    if re.search('CVSS V[0-9] [0-9]\.[0-9]', i.upper()) is not None:
      ver = re.search('CVSS V[0-9] [0-9]\.[0-9]', i.upper()).group()
      ver = re.sub('CVSS','',ver).strip()
      vul['CVSS']['version'] = ver
    elif re.search('CVSS V[0-9]', i.upper()) is not None:
      ver = re.search('CVSS V[0-9]', i.upper()).group()
      ver = re.sub('CVSS','',ver).strip()
      vul['CVSS']['version'] = ver
    elif re.search('CVSSVersion [0-9].[0-9]', i) is not None:
      ver = re.search('CVSSVersion [0-9].[0-9]', i).group()
      ver = re.sub('CVSS','',ver).strip()
      vul['CVSS']['version'] = ver
    elif re.search('CVSS Version [0-9].[0-9]', i) is not None:
      ver = re.search('CVSS Version [0-9].[0-9]', i).group()
      ver = re.sub('CVSS ','',ver).strip()
      vul['CVSS']['version'] = ver
    elif re.search('CVSSV[0-9]', i.upper()) is not None:
      ver = re.search('CVSSV[0-9]', i.upper()).group()
      ver = re.sub('CVSS','',ver).strip()
      vul['CVSS']['version'] = ver
    elif re.search('CVSS:[0-9]\.[0-9]', i.upper()) is not None:
      ver = re.search('CVSS:[0-9]\.[0-9]', i.upper()).group()
      ver = re.sub('CVSS:','',ver).strip()
      vul['CVSS']['version'] = ver
    elif re.search('CVSSd Version [0-9].[0-9]', i) is not None:
      ver = re.search('CVSSd Version [0-9].[0-9]', i).group()
      ver = re.sub('CVSSd ','',ver).strip()
      vul['CVSS']['version'] = ver
  # find the base score and temporal score
  if 'base score of ' in i:
    if re.search('base score of [0-9]{1,}\.[0-9]{1,}',i) is not None:
      s = re.search('base score of [0-9]{1,}\.[0-9]{1,}',i).group()
      s = re.sub('base score of ','',s)
      vul['CVSS']['base score'] = s
  if "temporal score of " in i:
    if re.search('temporal score of [0-9]{1,}\.[0-9]{1,}',i) is not None:
      s = re.search('temporal score of [0-9]{1,}\.[0-9]{1,}',i).group()
      s = re.sub('temporal score of ','',s)
      vul['CVSS']['temporal score'] = s
  # find the intrusions
  intru = '(result(s|ing)? in|causes?|allows?( attakers?)?) [A-Za-z ]+(,|.)'
  for s in i.split('.'):
    if re.search(intru, s) is not None:
      vul['Intrusion'] = s
  return vul

## 1.4 Get the Vulnerability Details

#### 1.4.1 Side Cases

In [20]:
# A function to extract vulneralbility information in a side case
def side_divider(i,cvss):
  flag = 0
  x = 0
  act = 0
  y = 0
  line_flag = 0
  begin = 0
  Vuls=[]
  vul = init()
  for j in read_file(i):
    # stop searching when the end divider appears
    gap = ['VULNERABILITY CHARACTERIZATION']
    if text_equal(j.upper(),gap):
      flag = 0
      if vul['CVSS']['version'] == '':
        vul['CVSS']['version'] = cvss
      try:
        vul['text'] = vul_text
      except:
        vul['text'] = ''
      Vuls.append(vul)
    if line_flag:
      begin = 1
      vul = init()
      vul['Type'] = re.sub(' CWE-[0-9]{3}$','',ty)
      vul['CWE'] = cwe
      line_flag = 0
    if flag:
      if j.upper() == 'VULNERABILITY OVERVIEW':
        continue        
      s = find_key(j,['\['])
      line = j[:s]
      # find the vulnerability type
      if check_line(line) and not find_sub(line,[': ']):
        try:
          complete_vul_text = vul_text
        except:
          pass
        vul_text = line
        line_flag = 1
        st = re.search('[A-Za-z]',line)
        s = 0
        if st is not None:
          s = st.start()
        ty = line[s:]
        cwe = find_cwe(i,j)
        if x:
          if vul['CVSS']['version'] == '':
            vul['CVSS']['version'] = cvss
          vul['text'] = complete_vul_text
          Vuls.append(vul)
        x = 1
      elif len(line.split())<=4 and not find_sub(line,[': ']):
        vul_text = line
        line_flag = 1
        st = re.search('[A-Za-z]',line)
        s = 0
        if st is not None:
          s = st.start()
        ty = line[s:]
        cwe = find_cwe(i,j)
        if x:
          if vul['CVSS']['version'] == '':
            vul['CVSS']['version'] = cvss
          vul['text'] = complete_vul_text
          Vuls.append(vul)
        x = 1
      elif begin:
        vul_text = vul_text + ', ' + j
        vul = process_vul(j,vul)
    # start searching when the start divider occurs
    sec = ['BACKGROUND']
    if find_sub(j.upper(),sec):
      act = 1
      flag = 1  
      y = 1
  return [x, Vuls]

In [21]:
# A function to extract vulneralbility information in anotehr side case
def new_side_divider(i,cvss):
  
  x = 0
  flag = 0
  act = 0
  y = 0 
  line_flag = 0
  begin = 0
  Vuls=[]
  vul = init()

  for j in read_file(i):  
    # stop searching when the ending divider occurs
    if j.title() == 'Exploitability':
      if vul['CVSS']['version'] == '':
        vul['CVSS']['version'] = cvss
      try:
        vul['text'] = vul_text
      except:
        vul['text'] = ''
      Vuls.append(vul)
      flag = 0
    # record when the first divider occurs
    if line_flag:
      begin = 1
      vul = init()
      vul['Type'] = re.sub(' CWE-[0-9]{3}$','',ty)
      vul['CWE'] = cwe
      line_flag = 0
    if flag:
      # skip the wrongly aligned line
      if j.upper() == 'VULNERABILITY OVERVIEW':
        continue
      s = find_key(j,['\['])
      line = j[:s]
      # find the subsection in vulneralbility details
      if check_line(line):
        try:
          complete_vul_text = vul_text
        except:
          pass
        vul_text = line
        line_flag = 1
        st = re.search('[A-Za-z]',line)
        s = 0
        if st is not None:
          s = st.start()
        ty = line[s:]
        cwe = find_cwe(i,j)
        if x:
          if vul['CVSS']['version'] == '':
            vul['CVSS']['version'] = cvss
          vul['text'] = complete_vul_text
          Vuls.append(vul)
        x = 1
      elif begin:
        vul_text = vul_text + ', ' + j
        vul = process_vul(j,vul)
    if j.upper() == 'VULNERABILITY DETAILS':
      act = 1
      flag = 1 

  return [x, Vuls]

In [22]:
def no_type(i,cvss):

  x = 0
  flag = 0
  act = 0
  begin = 0
  line_flag = 1
  Vuls=[]
  vul = init()

  for j in read_file(i):
    # stop searching when end divider occurs
    if find_sub(j.upper(),nt_gap):
      Vuls.append(vul)
      flag = 0
    if flag:
      if j.upper() == 'VULNERABILITY OVERVIEW':
        continue
      s = find_key(j,['\['])
      line = j[:s]
      if check_line(line):
        try:
          complete_vul_text = vul_text
        except:
          pass
        vul_text = line
        x = 1
        vul['Type'] = re.sub(' CWE-[0-9]{3}$','',line)
      else:
        try:
          vul_text = vul_text + ', ' + line
        except:
          vul_text = line
        vul = process_vul(line,vul)
    # start searching when start divider occurs
    if find_sub(j.upper(),nt_sec):
      act = 1
      flag = 1  
      y = 1
  try:
    vul['text'] = vul_text
  except:
    vul['text'] = ''
  Vuls.append(vul)
  return [x, Vuls]

In [23]:
def side_case_vul(i):
  # get the cvss score from the advisory as backup
  cvss = find_cvss(i)
  # process three kinds of side cases of vulneralbility
  result = side_divider(i,cvss)
  if not result[0]:
    result = new_side_divider(i,cvss)
    if not result[0]:
      result = no_type(i,cvss)
      return result[1]
    else:
      return result[1]
  else:
    return result[1]

#### 1.4.2 General cases

In [24]:
def vulner(i):
  # back up the cvss score
  cvss = find_cvss(i)

  flag = 0
  x = 0
  act = 0
  y = 0
  line_flag = 0
  begin = 0

  Vuls = []
  vul = init()

  for j in read_file(i):
    # skip when the next line is wrongly placed
    if act:
      act = 0
      if text_equal(j.upper(),vul_err):
        continue
    # stop collecting when the end occurs
    if (text_equal(j.upper(),vul_gap) or j == 'Exploitability') and flag:
      flag = 0
      if vul['CVSS']['version'] == '':
        vul['CVSS']['version'] = cvss
      try:
        vul['text'] = vul_text
      except:
        vul['text'] = ''
      Vuls.append(vul)
    # record when collection is initiated
    if line_flag:
      begin = 1
      vul = init()
      vul['Type'] = re.sub(' CWE-[0-9]{3}$','',ty)
      vul['CWE'] = cwe
      line_flag = 0
    if flag:
      s = find_key(j,['\['])
      line = j[:s]
      if check_line(line):
        try:
          complete_vul_text = vul_text
        except:
          pass
        vul_text = line
        line_flag = 1
        st = re.search('[A-Za-z]',line)
        s = 0
        if st is not None:
          s = st.start()
        ty = line[s:]
        cwe = find_cwe(i,j)
        if x:
          if vul['CVSS']['version'] == '':
            vul['CVSS']['version'] = cvss
          vul['text'] = complete_vul_text
          Vuls.append(vul)
        x = 1
      elif len(line.split())<=4:
        try:
          complete_vul_text = vul_text
        except:
          pass
        vul_text = line
        line_flag = 1
        st = re.search('[A-Za-z]',line)
        s = 0
        if st is not None:
          s = st.start()
        ty = line[s:]
        cwe = find_cwe(i,j)
        if x:
          if vul['CVSS']['version'] == '':
            vul['CVSS']['version'] = cvss
          vul['text'] = complete_vul_text
          Vuls.append(vul)
        x = 1
      elif begin:
        vul_text = vul_text +', ' + j
        vul = process_vul(j,vul)
    # start searching when starting divider occurs
    if text_equal(j.upper(),vul_sec):
      act = 1
      flag = 1  
      y = 1
  # push side cases to other functions
  if not x and y:
    vul = side_case_vul(i)
    return vul
  elif not x and not y:
    vul = []
    return vul
  else:
    return Vuls

## 1.5 Get General Information

In [25]:
# A function to get information from the text file
def get_info(i):

  Info = {}
  # get the title and the link
  Info['Title'] = i.split('.')[0]
  Info['Link'] = "https://www.us-cert.gov/ics/advisories/"+Info['Title']
  # get researcher 
  res_flag=0
  Info['Researcher']=''
  # get vulnerability information
  Info['Vulnerabilities'] = vulner(i)
  # get the vendor for the advisory
  Info['Vendor'] = get_vendor(i)

  for line in read_file(i):
    if res_flag:
      res_flag = 0
      Info['Researcher'] = line
    if re.search('researcher',line.lower()) is not None:
      s = re.search('researcher',line.lower()).start()
      if line[s:].lower() == 'researcher':
        res_flag = 1

  return Info

# 2 Affected Products


### 2.1 NLP processing

#### 2.1.1 Utilities

In [26]:
# utilities for NLP processing for illegal structures
# for strings in pname
alert_st = ['VERB','AUX','X','PROPN PROPN PROPN NOUN NOUN','DET ADJ NOUN PUNCT'] 
unwanted_st = ['VERB AUX VERB','AUX ADV VERB','AUX PART VERB','VERB SCONJ PART',
               'INTJ VERB','AUX VERB ADJ','VERB SCONJ DET','NOUN PUNCT PART',
               'PUNCT NOUN PUNCT X PUNCT','AUX VERB AD','AUX ADV','VERB DET NOUN',
               'PROPN AUX','AUX ADJ ADP','AUX SCONJ','PART VERB','ADV VERB ADP',
               'DET NOUN VERB ADP','AUX PART ADJ','VERB ADP DET PROPN PROPN',
               'AUX DET ADV','DET ADJ NOUN CCONJ','VERB PROPN PART','AUX NOUN ADP',
               'VERB VERB ADP DET','DET ADJ NOUN PUNCT','ADJ NOUN VERB VERB',
               'PROPN CCONJ PROPN VERB AUX NOUN']
unwanted_p = ['.com','.htm','Note']
# for strings in version
unwanted_v = ['versions including:','not impacted','which is','versions including']
unwanted_st_v = ['VERB ADV AUX VERB','DET AUX VERB PUNCT','SCONJ DET PROPN',
                 'AUX VERB PART','AUX NOUN PUNCT','^NOUN VERB PUNCT$','^NOUN AUX VERB$',
                 'VERB DET NOUN ADP','AUX AUX VERB SCONJ','NOUN AUX NOUN',
                 'NOUN VERB ADV AUX VERB','DET NOUN AUX','NOUN VERB ADV AUX ADJ',
                 'DET NOUN VERB AUX','NOUN VERB ADV PUNCT']
unwanted_p_v = ['Only','These pumps were','This does not apply','In ']
alert_st_v = ['AUX','X','VERB']

#### 2.1.2 Functions

In [27]:
# A function to generate a string of the sentence's POS tagging
def sent_struct(line):
  struct = ''
  txt = nlp(line)
  for t in txt:
    struct = struct + ' ' + t.pos_
  return struct

In [28]:
# A function to filter affect product information with certain features
def ap_filt(p):
  # filter side cases
  pname = p['Pname']
  if '(enabled with iBed Wireless)' in pname:
    return 1
  if 'respective Ethernet communication modules listed below:' in p['version']:
    return 1
  # clean the product name
  pname = pname.lstrip('·').rstrip(': ').rstrip(', ')
  pname = re.sub(' und ', ' and ', pname)
  pname = re.sub(' \(.+\):? ?$','',pname)
  pname = pname.rstrip(', and')
  pname = pname.rstrip(', with')
  pname = pname.rstrip('with')
  v = p['version']

  # get the structure of product name and version
  sent_ori_stct = sent_struct(pname)
  sent_stct_v = sent_struct(v)
  # examine whether the string may be useless according to structure
  st_flag = find_sub(sent_ori_stct,alert_st)
  st_flag_v = find_sub(sent_stct_v,alert_st_v)
  # examine whether the string is useless according to structure
  st_flag_2 = find_sub(sent_ori_stct,unwanted_st)
  st_flag_2_v = find_sub(sent_stct_v,unwanted_st_v)
  # examine whether the string is useless according to keywords
  p_flag = find_sub(pname,unwanted_p)
  p_flag_v = find_sub(pname,unwanted_p_v)
  v_flag = find_sub(v,unwanted_v)
  # calculate the final flag value
  flag_p = st_flag*(st_flag_2+p_flag+v_flag)
  flag_v = st_flag_v*(st_flag_2_v+p_flag_v)
  flag = flag_p+flag_v
  # add a side case
  if flag or pname=='Note - Experion PKS':
    return 0
  else:
    return 1

### 2.2 Structured Information

#### 2.2.1 Utilities

In [29]:
# Utilities for get_div
all_override = ['all versions prior','versions prior',': application versions',
                ': All versions', ': all versions',', all versions',
                ': All version',')  firmware version',') all versions',
                ' all versions',': All firmware',': application versions',
                ': Firmware V',': All Versions','All models','– All Versions',
                ' (Firmware Version','TCSEFEC','^QJ71E71','All Models',
                'running firmware versions prior to','^MV420','^MV440',
                '^750-[0-9]{3,} firmware versions','(All Revisions)',
                '^CPE[0-9]{3}','^CRU[0-9]{3}','^CPL[0-9]{3}',
                '(End of Life; Upgrade to CPE330)','3\.1 through 3\.3\.3']
div_firmware = ['part n° 5.02','firmware version ']
div_special = ['Dev.Rev.','Rev.','Rev ','Dev.Rev ']
div_me = [': ME prior to V']
div_kwds = ['(firmware|software) version','version','prior to ','revisions? ',
          'all (firmware |software )?versions?',' series .+[0-9]+',
      '( ver( |\. )?|a|v( |\.)?| \(?r)?[0-9]+\.([0-9]+\.?)+',  
      'back to', ' editions','firmware prior ','releases? ', 'running',
      '( ver( |\.)?|v( |\.)?| \(?r)[0-9]+', ' 20[0-1][0-9]( |,|/.)',
      'serial numbers?','operating on ','all devices? ',' produced between ',
      'wlan version ','manufactured before ','\(shipped between',
      ' up to and including ',' all windows',' older than',' \(all model',
      ' model ',' all current and older','mp[0-9]\.[0-9]','\(build','^XL1000C',
      '^CPR9','^NC85','^NET Firmware','^ION','^PM5XXX','^OnCellG','^AWK-',
      '^WAC-','^TAP-','^All ILC','^XLWeb','^1763-L','^750-820',' [0-9]\.x ',
      '^762-300','^1783-SAD','^2017 Update 2 and prior',', Models?',
      ': article number','^HEN[0-9]{5,6}L?','^Security Notification SN',
      '^H(B|E|4|2|P)(D|W)(3|8|2|4)P(R|C|E|1).{0,3}','^v12\.001 and prior',
      '^CP(E|U|L)(1|3|4)[0-3](0|2|5)','^Series PFC(1|2)00',
      '^750-8[0-9][2-3], 750-83[1-2]','^v21\.001 and prior',
      'Product ID/Reorder number:','UDI/Model/NDC number:',', released',
      'using Hi.+ version 0',', between','^1SAP1','^1TNE968','^TLXCD',
      ', all current and older version']
# Utilities for pull_head
# for pname
patterns_p = [' affects .+ listed', 
              'the sensys networks traffic sensors, .+ and .+,',
              'the affected .+ version', ' (following|all) .+ appliances',
              ' of .+?(,|are|is|:) ?', ' (following|all) .+ modules',
              ' (following|all) .+ (versions?|products?|are|models?)']
sub_p = [' of ',' are',' is',':',' following ', 'all ', '^the ','listed',
         ' products and versions$','affected',' affects',
         '(^hisecos)? (versions?|products?|models?|modules?)']
# for version         
patterns_v = ['distributed prior to .+, are','using .+ and lower ', 
              'with firmware .+( is| are|$|:)',
              'running firmware versions prior to.+ of', 
              ' all versions prior to .+ of','prior to version .+?of ',
              'version .+ and earlier are affected',
              'to the following products using software.+',
              ' of .+?(,|are|is|:) ?',' all versions? ', 
              'firmware version [A-Za-z0-9]+ (for|of|,|is|are|and)',
              '(version)? [0-9\.]+ and prior for the following']
sub_v = [' for the following', ' for ',' of ',':',' is',' are',' affected',
         'products using software',' of$','distributed',', regardless','with']

#### 2.2.2 Functions

In [30]:
# A function to pinpoint the product name and version in sentences
def get_div(line, keys = []):
  if 'TIA Portal V13 (including WinCC Professional Runtime):' in line:
    return len(line)
  elif re.search(' for:$',line) is not None:
    return 0
  line = line.rstrip('.')
  vers_flag = re.search(' versions [0-9]\.[0-9]{2,}', line)
  # different occasions relate to different keywords package
  if keys != []:
    kwds = keys 
  elif find_sub(line,div_firmware,1):
    kwds = ['firmware version ']
  elif re.search('^(V|v)[0-9]+(\.[0-9])?',line) is not None:
    kwds = ['^(V|v)[0-9]+\.[0-9]']           
  elif find_sub(line, div_special):
    kwds = div_special
  elif re.search('[0-9]\.[0-9]u[0-9]',line) is not None:
    kwds = ['version','all versions']
  elif find_sub(line, all_override) and vers_flag is None:
    kwds = all_override
  elif find_sub(line, div_me):
    kwds = [' prior to']
  else:
    kwds = div_kwds
  div = len(line)
  # use the smallest index as the divider
  for i in kwds:
    if kwds == all_override:
      i = re.sub('\(','\(',i)
      i = re.sub('\)','\)',i)
    if re.search(i.lower(), line.lower()) is not None:
      if (i == ' series .+[0-9]+' or i == ' model ') and div != len(line):
        break
      new_div = re.search(i.lower(), line.lower()).start()
      if new_div < div:
        div = new_div
  return div

In [31]:
# A function to extract information from lines in the head of a section
def pull_head(line, type_is):
  forall=''
  # to extract product name
  if type_is == 1:
    pattern = patterns_p
    subs = sub_p
  # to extract version
  elif type_is == 0:
    pattern = patterns_v
    subs = sub_v
  # find certain patterns in a string
  for i in pattern:          
    if re.search(i, line.lower()) is not None:
      forall = re.search(i, line.lower()).group()
      if re.search(i, line) is not None:
        forall = re.search(i, line).group().lower()
      break
  # clear noise in the string
  for s in subs:
    forall = re.sub(s,'',forall)
  if not type_is:
    forall = forall[get_div(forall):]
  forall = forall.strip()
  forall = re.sub('\(','\(',forall)
  forall = re.sub('\)','\)',forall)
  if re.search(forall,line.lower()) is None:
    return 0
  s = re.search(forall,line.lower()).start()
  e = re.search(forall,line.lower()).end()
  result = line[s:e]
  return result

### 2.3 Special Cases

#### 2.3.1 Utilities

In [32]:
# utilities for one_line
kwrds_ol_verb = [' affects? ',' impacts? ',' reports? that ']
to_sub_ol_kwds = [' are ',' software products ']
# utilities for exceptions
kwds_div = ['less than','equal to',', all versions.','prior to',
                'using NTP service','using ',': All versions']
kwds_find = ['less than','equal to',', all versions.',
             'using NTP service','All products using ',
             '0: All versions','Any DTM written']

#### 2.3.2 Functions

In [33]:
# A function to process affected products section with only one line
def one_line(line):
  # find the smallest index to divide
  if find_key(line.lower(),kwrds_ol_verb) != len(line):
    div = 0
    for k_verb in kwrds_ol_verb:
      new_div = re.search(k_verb, line.lower())
      if new_div is not None:
        new_div = new_div.end()
        if new_div > div:
          div = new_div
    line = line[div:]
  # extract information from generalizing sentence
  pin = re.search('all .+ of .+ (are )?(.+)?',line.lower())
  if pin is not None:
    s = pin.start()
    e = pin.end()
    line = line[s:e]
  # remove noise from certain sentences
  div = find_key(line,to_sub_ol_kwds)
  line = line[:div]
  line = re.sub('(a|A)ll (known )?versions? of (the)?','', line).strip()
  # remove 'all' at the beginning of a string
  if line[:4].lower() == 'all ':
    line = line[4:]
  return line

In [34]:
# A function to process sentences using 'all of' structure
def all_of(n, lines, aff_prod = []):
  cut = lines[n].split(' are')[0]
  nxt_cut = ''
  # examine whether the sentence should be processed
  if n < len(lines)-1:
    nxt_cut = lines[n+1].split(' are')[0]
  aff_prot = {'Pname':cut,'version':'0'}
  p_flag = ap_filt(aff_prot)
  # stop processing if lines don't contain information
  if not p_flag and not len(aff_prod):
    return ['']
  # pre-process lines with function one_line
  p = one_line(lines[n])
  # find seperate product names in following lines
  if re.search(' versions? of ', nxt_cut) is not None:
    temp = n
    while re.search(' versions? of ', cut) is not None:
      p = re.sub('of ','',re.search('of .+',cut).group())
      v = re.search('back to.+',p)
      if v is not None:
        v = v.group()
        p = re.sub(v,'',p)
      else:
        v = 'All versions'
      temp += 1
      cut = lines[temp].split(' are')[0]
      aff_prod.append({'Pname':p,'version':v})
    return aff_prod
  # collect product information after related vulnerablities
  if 'CVE' in cut:
    p = lines[n].split(':')[1]
    p = re.sub('of ','',re.search('of .+( |$)',p).group())
    v = p[get_div(p):]
    p = p[:get_div(p)]
    aff_prod.append({'Pname':p,'version':v})
    temp = n+1
    nxt = lines[temp].split(':')[1]
    while temp < len(lines):
      nxts = nxt.split(' and ')
      if not len(nxts)-1:
        nxts = nxts[0].split(',')
      for i in nxts:
        aff_prod.append({'Pname':i,'version':''})
      temp += 1
      if temp < len(lines):
        nxt = lines[temp].split(':')[1]
    return aff_prod
  # extract product information from the sentence
  elif 'version' in p.lower():
    to_sub = p
    if re.search(' of.+',p) is not None:
      p = re.sub(' of ','',re.search(' of.+',p).group())
      v = re.sub(p, '', to_sub)
    if 'are affected' not in lines[n] or 'are affected' not in p:
      keys=['prior to','using']
      v = lines[n][find_key(lines[n],keys):].split(' are')[0]
      p = re.sub(v,'',p)
    return {'Pname':p,'version':v}
  else:
    v = 'All Versions'
  # extract seperate version in following lines
  if ':' == p[-1]:
    temp = n+1
    nxt = lines[temp]
    while 'versions of' not in nxt:
      v = nxt
      if '(' in nxt:
        aff_prod[-1]['version'] = aff_prod[-1]['version'] + nxt
        temp += 1
        if temp >= len(lines):
          break
        nxt = lines[temp]
        continue
      aff_prod.append({'Pname':p,'version':v})
      temp += 1
      if temp >= len(lines):
        break
      nxt = lines[temp]
    if temp < len(lines):
      all_of(temp, lines,aff_prod)
    return aff_prod
  return {'Pname':p,'version':v}

In [35]:
# A function to extract information from the rest side cases
def exceptions(n, lines):
  line = lines[n]
  # skip 'not affected' information
  if 'not affected' in line.lower():
    return 1
  else:
    line = lines[n]
    # divide certain sentences to extract information
    if find_sub(line,kwds_find):
      d = get_div(line, kwds_div)
      p = line[:d]
      v = line[d:]
      return (1,{'Pname':p,'version':v})
    # extract information using function one_line
    elif re.search('All [A-Za-z ]+ are .+ed', line) is not None:
      p = one_line(line)
      v = ''
      return [1,{'Pname':p,'version':v}]
    # process side cases with 'with' in it
    elif 'with:·' in line:
      output = []
      line = line.split('with:·')[1]
      while n < len(lines):
        line = lines[n].split('with:·')[-1]
        d = get_div(line)
        p = line[:d]
        v = line[d:]
        output.append({'Pname':p,'version':v})
        n += 1
      return [2, output]
    # process a specific side case
    elif re.search('^DirectLogic DL', line) is not None:
      p = line
      n += 1
      output = []
      v = lines[n]
      while re.search('^DirectLogic DL', v) is None:
        output.append({'Pname':p,'version':v})
        n += 1
        if n == len(lines):
          break
        v = lines[n]
      return [3, output]
    else:
      return 1

### 2.4 Extract Product Info

#### 2.4.1 Utilities

In [36]:
# utilities for getting affected products
special = ['Dev.Rev.','Cond.','Rev.','Ing', 'ver.',
           'Rev','Dev.Rev','Cond','ver']
fisrt_sign = ['following','affected products','products affected:','products:',
              ' for profinet io:', 'reports that the vulnerability affects',
              'the affected codesys runtime version is part of']
stct_unexpected = ['VERB','AUX']
void_line = ['.','']
G_cases = ['G-Cam:','G-Code:']

#### 2.4.2 Functions

In [37]:
# A function to extract affected products' information
def get_ap(i):

  v_occupied = 0
  
  ap = []
  pot = ''
  list_adv = collect_lines(i)[1]
  if len(list_adv) == 0:
    return []
  num_lines = len(list_adv)
  j = 0

  # process side cases with version information in the end
  if 'For the listed products' in list_adv[-1]:
    vers = list_adv[-1]
    vend = list_adv[0]
    v = re.search('all.+[0-9]',vers).group()
    ven = re.search('following.+models',vend).group().split()[1]
    for line in list_adv[1:-1]:
      for p in line.split(', '):
        ap.append({'Pname':ven+' '+p.lstrip('·'),'version':v})
    return ap
  # process cases with only one line
  if num_lines == 1:
    l = list_adv[0]
    if re.search(' versions? of ', l.lower()) is not None and 'InTouch' not in l:
      result = all_of(j, list_adv, [0])
      return [result]
    d_ori = get_div(l)
    p = l[:d_ori]
    v = l[d_ori:]
    if v in void_line:
      p = pull_head(l, 1)
      v = ''
      if not p:
        return []
    if find_sub(sent_struct(p),stct_unexpected):
      ol = one_line(l)
      d = get_div(ol)
      p = ol[:d]
      v = ol[d:]
      if v in void_line:
        v = 'All Versions'
    ap.append({'Pname':p,'version':v})
    return ap

  first_line = 0
  while j < num_lines:
    
    # remove noise strings in the line
    ori_l = list_adv[j].rstrip('.').lstrip('·')
    if '•\xa0\xa0 \xa0' in ori_l:
      ori_l = ori_l[5:]
    elif '\xa0 \xa0 \xa0 \xa0\xa0 ' in ori_l:
      ori_l = ori_l[9:]
    elif '\xa0 \xa0 \xa0 \xa0 \xa0' in ori_l:
      ori_l = ori_l[9:]
    l = ori_l.lower()
    # process side cases with useless strings
    if pot == 'special-1':
      p = re.sub('(All | are affected )','',ori_l)
      v = ''
      ap.append({'Pname':p,'version':v})
      return ap
    # skip side cases with certain information
    if 'For the Alaris Gateway Workstation' in ori_l:
      j+=1
      continue
    # strip phrases in a sentence
    if ori_l.split()[-1] == 'Versions' and ', All Versions' not in ori_l:
      ori_l = re.sub('Versions','',ori_l)
      l = ori_l.lower()
    # skip lines as subsection divider
    if re.search('4\.1\.[0-9]\xa0',ori_l) is not None:
      j+=1
      pot = ''
      first_line = 1
      continue
    # skip lines that don't initiate rightly
    if not first_line and not find_sub(l,fisrt_sign):
      if 'This vulnerability impacts' in ori_l:
        ori_l = ori_l.split('impacts ')[1]
        l = ori_l.lower()
        first_line = 1
        pot = 'special-1'
        pass
      # process exceptions
      else:
        flag_excpt = exceptions(j, list_adv) 
        if flag_excpt == 1:
          j+=1
          continue
        elif flag_excpt[0] == 1:
          ap.append(flag_excpt[1])
          j+=1
          continue
        elif flag_excpt[0] > 2:
          ap = ap + flag_excpt[1]
          j += len(flag_excpt[1])+1
          continue
        else:
          return flag_excpt[1]
    # skip lines with 'not affected' information
    if 'not affected' in l:
      j+=1
      first_line = 0
      continue
    if find_sub(l, fisrt_sign):
      # skip when the report starts talking about vendors
      if re.search('the following vendor companies:$', l) is not None:
        j+=1
        continue
      # skip when MLFBs are to be referred
      elif 'with the following MLFBs' in ori_l:
        first_line = 0
        j+=1
        continue
      pot = ori_l
      first_line = 1
    # process lines using all_of function
    elif re.search(' versions? of ', l) is not None and 'InTouch' not in ori_l:
      result = all_of(j, list_adv, [])
      if type(result) == dict or result == ['']:
        if result != ['']:
          ap.append(result)
        j += 1
        continue
      else:
        return result

    # process cases that are more general 
    else:
      d_ori = get_div(ori_l)
      p = ori_l[:d_ori]
      v = ori_l[d_ori:]
      # process side cases with 'G-Cam:' and 'G-Code:'
      if find_sub(p, G_cases): 
        p = ''
        d_ori = 0
      # complete lines that are mistakenly split
      if ori_l.split()[-1] in special:
        j+=1
        if ori_l[-1] != '.':
          ori_l = ori_l + '.'
        ori_l = ori_l + list_adv[j]
        d = get_div(ori_l)
        p = ori_l[:d]
        v = ori_l[d:]
      # extract continuously in cases with version/pname only appear in the first line
      elif (d_ori == len(l) or d_ori == 0) and j < len(list_adv)-1:
        d = d_ori
        temp = j+1
        now = j
        next_line = list_adv[temp].rstrip('.').lstrip('·').strip('•\xa0\xa0 \xa0')
        mark = len(next_line)-len(next_line)*(d>1)
        # stops when the line is not pure pname/version
        while get_div(next_line) == mark:
          if d == 0:
            p = next_line
          elif d == len(l):
            v = next_line
          if re.search(' for:$',l) is not None:
            v = ori_l.split(' of')[0]
          if p == 'This notification can be found at the following location:':
            next_line = ''
            continue
          # process side cases with CVE information
          if 'CVE-2019-1093' in p:
            if 'SIPROTEC 5 with CPU variants CP200' in ap[-1]['Pname']:
              p = ap[-1]['Pname']
              if ap[-1]['version'] == '':
                ap.pop()
          aff_prod = {'Pname':p,'version':v}
          aff_flag = ap_filt(aff_prod)         
          if aff_flag:
            ap.append({'Pname':p,'version':v})
          temp+=1
          if temp >= len(list_adv):
            break
          next_line = list_adv[temp].rstrip('.').lstrip('·').strip('•\xa0\xa0 \xa0')
          mark = len(next_line)-len(next_line)*(d>1)
        j = temp-1
        # look for information in the first lines
        if now == j:
          # look for product name
          if d_ori == 0:
            p = pull_head(pot, 1)
          # look for version
          elif d_ori == len(l):
            v = pull_head(pot, 0)
          if p != '' and v != '':
            # process side cases with certain version format
            sc_sap = re.search('^1SAP1',v)
            sc_tne = re.search('^1TNE968',v)
            if sc_sap is not None or sc_tne is not None:
              p = pull_head(pot,1) + ' ' + v
              v = pull_head(pot,0)
            ap.append({'Pname':p,'version':v})
      # process lines at the end of the document with partial information
      elif (d_ori == len(l) or d_ori == 0) and j == len(list_adv)-1:
        if d_ori == 0:
          p = pull_head(pot, 1) # pull head
        elif d_ori == len(l):
          v = pull_head(pot, 0)
      # skip 'not affected' lines
      else:
        if 'not affected' in l:
          j+=1
          continue
      # reprocess lines with wrong division
      if v[:4] == 'V11 ' and 'all version' in v.lower():
        p = p + 'V11'
        v = 'All versions'
      elif v[:3] == 'V3 ' or v == 'V3' or v == 'V3,' or v[:4] == 'V11 ':
        p = p + v
        v = pull_head(pot, 0)
      elif re.search('^(V|v)[0-9]',p) is not None and 'all version' in p.lower():
        v = p
        p = ap[-1]['Pname']
        if ap[-1]['version'] == '':
          ap.pop()
      # supplement lines missing information
      if 'WEB’log' in pot:
        p = 'WEB’log ' + p
      aff_prod = {'Pname':p,'version':v}
      # filter lines with certain POS taggings or text
      # st = sent_struct(p)
      p_flag = ap_filt(aff_prod)
      if p == '\t':
        ap[-1]['version'] = v
        p_flag = 0
      if aff_prod not in ap and p_flag:
        ap.append(aff_prod)
    j+=1
  return ap

### 2.5 Version conversion


#### 2.5.1 Utilities

In [38]:
# utilities for version conversion
words_of_interest = ['prior', 'earlier', 'through', 'before','up to','between',
                     'after', 'back to','older than','prior to','including',
                     'excluding', 'from','or lower','or older','or higher',
                     'and previous versions','and later','and newer',
                     'and lower','and below','and greater','and above',
                     'and potentially prior','and previous','and older',
                     'all versions earlier than','and all previous versions',
                     'or earlier','all subsequent minor releases','and prior',
                     'and possibly earlier versions','and any previous version']

sub_in_v_c = [' ?all versions earlier than ?',
              ' ?and all subsequent minor releases ?',
              ' ?and possibly earlier versions ?',' ?and any previous version ?',
              ' ?and previous versions ?', ' ?and all previous versions ?,?', 
              'devices that include the ', ' ?and possibly earlier versions ?', 
              ' ?back to ?', ' ?or earlier ?', ' ?(A|a)ll (?!firmware )versions? ?,?',
              '^(v|V)ersions(:·| )?', ' ?(A|a)ll ', ' ?shipped between ?', 
              ' (excluding|including) ?', ' ?up to ?', ' ?(or )?prior ?',
              ' ?older than ?', ' ?and potentially prior releases ?',
              ' ?and (all )?prior ?',' ?newer ?',' ?(or )?lower ?',' ?below ?',
              ' ?later ?',' ?(or )?higher ?',' ?above ?',' ?previous ',
              ' ?greater ?', ' ?(or )?older ?',' ?than ?',' and ?',
              ' ?prior ?(to )?',' ?(or )?earlier ?(than )?', ' ?before ?',
              ' ?(v|V)ersions(,|:·|;| |, and| )?$',' ?possibly ?',' of the ']

# keywords related to smaller/smaller or equal to
prece = ['and earlier','prior to','before','up to','and prior','and below',
          'and all prior','or lower','and previous versions','and lower','prior',
          'and potentially prior','and previous','all versions earlier than',
          'and all previous versions','or earlier','and possibly earlier versions',
          'lower than','and any previous version']
prece_incl = ['and earlier','up to','and prior','and below','and potentially prior',
              'and all prior','or lower','and previous versions','and lower',
              'and previous','and all previous versions','or earlier',
              'and possibly earlier versions','including','and any previous version']
# keywords related to larger/larger or equal to
subse = ['and all subsequent minor releases','older than','or higher','and later',
          'and newer','back to','and greater','and above','and older']
subse_incl = ['and all subsequent minor releases','or higher','and above','and later',
              'and newer','and greater','and older','including','back to']

In [39]:
# unique utilities for cut_useless
skip_cut_kwds = ['between','MLFB 6SR']
add_to_kwds = ['including','Patch']
# unique utilities for get_ver
move_all_kwds = ['between','starting at ','through']
two_side_kwds = ['including','excluding']
# unique utilities for ver_con
extract_st = ['ADV VERB DET','ADV VERB PRONP']
kwrds_unwanted = ['.xlsx','not affected','.com','CVE',
                  'News updates','Wind River System products']
void_head = ['Quantum','Premium','M340']
prod_conj = [' and ', ' of ']
key_verb = ['impacts','affects']

#### 2.5.2 Functions

In [40]:
# A function to remove useless information from version
def cut_useless(ver):
  if re.search('\(.+\),?$',ver) is not None:
    out = ver
    return out
  elif re.search('^750-8[0-9][2-3], 750-83[1-2]',ver) is not None:
    out = ver
    return out
  elif find_sub(ver,skip_cut_kwds):
    out = ver
    return out
  out = ''
  for i in ver.split(','):
    if i == ' all versions affected':
      continue
    i = i + ' '
    if find_sub(i,words_of_interest):
      to_sub = i
      for s in sub_in_v_c:
        to_sub = re.sub(s,'',to_sub)
      if re.sub(' ','',to_sub) == '':
        out = i + ' ' + out[1:]
      elif 'and all previous versions' in i:
        i = re.sub('and all previous versions','',i)
        out = 'and all previous versions' + ' ' + out + ', ' + i 
      else:
        if find_sub(i,add_to_kwds):
          out = out + ', ' + i
        else:
          out = out + ' ' + i
    elif get_div(i) <= 1 or 'and' in i[:4]: 
      out = out+', '+i
    elif 'or' in i[:3] or '&' in i:
      out = out+', '+i
    elif re.search('^( [0-9]+| released)',i) is not None:
      out = out+', '+i 
    elif re.search('^( ?G[0-9][0-9]| Service Pack| Patch) ?',i) is not None:
      out = out+', '+i 
  return out.lstrip(', ')

In [41]:
# A function to locate where the inequality is
def pinpoint_sign(vers, rule, mark):
  parts = []
  slices = vers.split(', ')
  for i in slices:
    part = i
    if find_sub(re.sub('\(.+\)','',i), rule):
      if i == 'and any previous version':
        part = ', '.join(slices[:-1])
        parts = []
      part = mark + re.sub('^, ','',part)
    parts.append(part)

  return ', '.join(parts)

In [42]:
# A function to get the value of versions
def get_ver(vers):
  # Strip off irrelevant words
  vers = re.sub(',  including',', containing', vers)
  if find_sub(vers,['<','>','=']):
    vers = re.sub('^((A|a)ll )?(V|v)ersions ?','',vers)
  if find_sub(vers.lower(),words_of_interest):
    # Strip off 'are affected' part at the end of a string
    if ' are ' in vers:
      vers = vers.split(' are ')[0]
    if 'all versions through' in vers.lower():
      value = '<= ' + re.sub('all versions through','',vers.lower())
    elif vers == 'all current and older versions':
      value = vers
    elif re.search('after .+ and before', vers) is not None:
      value = vers[re.search('after .+ and before', vers).start():]
    elif find_sub(vers.lower(),prece) and find_sub(vers.lower(),subse):
      value = re.sub('((A|a)ll )?(v|V)ersions','',vers)
    elif find_sub(vers.lower(),move_all_kwds):
      value = re.sub('((A|a)ll )?(V|v)ersions ?','',vers)
    elif find_sub(vers.lower(),two_side_kwds,1):
      value = vers
    # Transform version string to general expressions with inequality
    elif find_sub(vers.lower(),prece):
      mark = '< '
      if find_sub(vers,prece_incl) and re.search('\(including ', vers) is None:
        mark = '<= '
      if ', ' in vers:
        vers = pinpoint_sign(vers, prece, mark)
      else:
        vers = mark + vers
      for i in sub_in_v_c:
        vers = re.sub(i,' ',vers)
      value = re.sub('^<= (v|V)ersions(:·| )?','<= ',vers)
      value = re.sub('^< (v|V)ersions(:·| )?','< ',value)
    elif find_sub(vers.lower(),subse):
      mark = '> '
      if find_sub(vers,subse_incl) and re.search('\(including ', vers) is None:
        mark = '>= '
      if ', ' in vers:
        vers = pinpoint_sign(vers, subse, mark)
      else:
        vers = mark + vers
      for i in sub_in_v_c:
        vers = re.sub(i,' ',vers)
      value = re.sub('^>= (v|V)ersions(:·| )?','>= ',vers)
      value = re.sub('^> (v|V)ersions(:·| )?','> ',value)
    else:
      value = vers
  elif vers == 'Versions:·3.5':
    value = re.sub('Versions:·','',vers)
  else:
    value = vers
  if '(' not in value and value[0]!='(':
    value = value.rstrip(')')
  # Clean the remaining noise from value string
  value=re.sub(' +',' ',value)
  value=re.sub('\.\.\.','to',value)
  value=re.sub(' ,',',',value)
  return value

In [43]:
# A function to add version value to affected products' information
def ver_con(i):
  output = []
  ap = get_ap(i)
  
  for p in ap:
    store = []
    line = p['Pname']+p['version']
    
    # skip certain product name which is actually sub-section divider
    if p['Pname'] in void_head:
      continue
    # store another affected product in the line 
    if ': Catapult ' in p['version']:
      store.append(p['version'].split(':')[1])
      p['version'] = p['version'].split(':')[0]
    # clean product names with certain errors
    elif re.search('^(Ind|D)irectly affected:',p['Pname']):
      p['Pname'] = p['Pname'].split(':')[1]
    elif p['Pname'] == 'and':
      p['Pname'] = ap[0]['Pname']
    elif re.search('all .+ versions (are affected)?.?$',line.lower()) is not None:
      p['Pname'] = re.sub('(A|a)ll|versions?|are affected','',line)
      p['version'] = 'All versions'
    elif find_sub(p['Pname'],kwrds_unwanted) or 'products:' == p['Pname'].split()[-1]:
      continue
    elif find_sub(p['version'],[': All versions',': (End of Life; Upgrade to CPE330)']):
      p['Pname'] = p['Pname']+' '+p['version'].split(':')[0]
      p['version'] = p['version'].split(':')[1]
    elif 'Version' in p['Pname']:
      d = get_div(p['Pname'],[' Version'])
      p['version'] = p['Pname'][d:]
      p['Pname'] = p['Pname'][:d]
    p['Pname'] = re.sub('(\(CPR9|versions)$','',p['Pname'])
    # skip when pname is void or the sentence is a generalizing one
    if p['Pname'] == '':
      pass
    elif p['Pname'][-1] == ':' and p['version']=='':
      continue
    elif re.search('^Ships with ', p['Pname']) is not None:
      continue
    # clean product name information with unnecessary words
    Pname = p['Pname']
    if Pname[:4].lower() == 'all ':
      Pname = Pname[4:]
    elif Pname[:4].lower() == 'any ':
      Pname = Pname[4:]
    while polish(Pname)!=Pname:
      Pname = polish(Pname)
    if 'listed in the mitigations' in Pname:
      Pname = Pname.split(' prior to applying the patch')[0]
    if ' Works together with other products listed here' in Pname:
      Pname = Pname.split(':')[0]
    p['Pname'] = Pname.split(', which ')[0]
    # clean version information
    version = cut_useless(p['version'])    
    ori_ver = p['version']
    if re.search('4\.1\.[0-9]\xa0\xa0 \xa0',p['version']) is not None:
      version = ''
    if ') with CPU variants CP300 and CP100' in version:
      version = version.split(')')[0]
    version = version.split(' are affected')[0]
    version = version.split(' is affected')[0]
    version = version.split(' is impacted')[0]
    version = version.split(' could be affected')[0]
    version = re.sub('p;rior','prior',version)
    version = re.sub('\xa0',' ',version)
    while polish(version)!=version:
      version = polish(version)
    if version == 'versions including':
      continue
    p['version'] = version
    # get version value from function get_ver
    if p['version'] != '' and p['version'].lower()!='all versions':
      vers = get_ver(p['version'])
      if '):' in version:
        version = version.split('):')[0]
      p['version'] = vers
    # complete version value that are wrongly split
    if p['version'] == 'versions':
      p['version'] = 'All Versions'
    # process another product information if there's any
    if store != []:
      cont = store[0]
      d = get_div(cont)
      pname = cont[:d]
      ver = cont[d:]
      vers = get_ver(ver)
      ap.append({'Pname':pname, 'version':vers})
    # process certain product information if pname is in version
    if find_sub(p['version'],prod_conj,1):
      p['Pname'] = vers.split(' of ')[1]
      p['version'] = vers.split(' of ')[0]
    # skip invalid information
    if version == '2015/01/20-001 addresses':
      continue
    if p['Pname'] == 'product and versions':
      p['Pname'] = output[0]['Pname']
    # extract production information from pname with certain POS format
    p_st = sent_struct(p['Pname'])
    if find_sub(p_st,extract_st):
      key = find_key(p['Pname'],key_verb)
      p['Pname'] = re.sub('(impacts|affects) the','',p['Pname'][key:])
    p_flag = re.search('^R4[0-9]',p['Pname']) is not None
    v_flag = re.search('^R4[0-9]',p['version']) is not None
    if p_flag*v_flag:
      prod = re.sub('R|[0-9]\.?|release|and','',p['version']).strip()
      p['version'] = re.sub((prod+' '),'',p['Pname'] + ', '+ p['version'])
      p['Pname'] = prod
    # clean the version text
    p['version'] = re.sub(',,','',p['version'])
    cell = {'Pname':p['Pname'],'version':p['version'],
            'version text':ori_ver}
    if cell not in output:
      output.append({'Pname':p['Pname'],'version':p['version'],
                     'version text':ori_ver})
  return output

# 3 Metrics Processing


### 3.1 Vulnerability Metrics

#### 3.1.1 Utilities

In [44]:
# utilities for collecting key lines
ui_special = ['user interaction','user ','social engineering']
av_special = ['local access','physical access']
av_special_more = ['remote','local','adjacent']
ac_special = ['skill',' difficult ']
av = ['physical access','remote','local','adjacent','network access']
ac = ['skill',' difficult ','easy','crafting','simple']
ui_1 = ['social engineering','triggered']
ui_2 = [' or ',' user ']
ui_3 = ['user interaction','user ']
# utilities for extracting metrics value
met_val = {'UI':{'can be exploited':'None'},
           'AV':{'local':'Local','remote':'Network','adjacent':'Adjacent',
                 'physical':'Physical','network':'Network'},
           'AC':{' to medium':'',' to high':'',' to moderate':'','-medium':'',
                 '-high':'','-moderate':'', 'moderate ':'case','medium':'case', 
                 'difficult':'High', 'low':'Low','High':'High',
                 'moderately difficult':'High','easy':'Low', 'simple':'Low', 
                 'more skilled':'High', 'take some effort':'High'}}
metric_sc = ['some','both','not ?.+(remotely| remotely)',
             'except for','(one|three) of ']         
typical = ['^(These|This|The) vulnerabilit(ies|y) ','^(The|An) attacker',
           '^All the vulnerabilities','^Physical network','^This exploit',
           '^In order to','^Exploitation of']                   

#### 3.1.2 Functions

In [45]:
# A function to check whether a sentence have more than one metrics
def see_sides(line):
  result = []
  for i in line:
    if find_sub(i.lower(), ui_special):
      result.append(1)
    elif find_sub(i.lower(), av_special):
      result.append(3)
    elif find_sub(i.lower(), av_special_more):
      result.append(2)
    elif find_sub(i.lower(), ac_special):
      result.append(3)
  r = len(set(result)) > 1
  return r

In [46]:
# A function to divide lines with metric information
def split_lines(r):
  output = []
  if '; ' in r:
    output = output + r.split('; ')
  elif ' via ' in r.lower():
    if see_sides(r.lower().split(' via ')):
      output = output + r.lower().split(' via ')
    else:
      output.append(r)
  elif ' but ' in r.lower():
    if see_sides(r.lower().split(' but ')):
      output = output + r.lower().split(' but ')
    else:
      output.append(r)
  elif ' and ' in r.lower():
    if see_sides(r.lower().split(' and ')):
      output = output + r.lower().split(' and ')
    else:
      output.append(r)
  else:
    output.append(r)
  return output

In [47]:
# A function to indentify sentences that state the metrics from the file
def properties(line, sort):
  p = {}
  # process text within sections
  if sort:
    for i in line:
      j = split_lines(i)
      for k in j:
        if find_sub(k.lower(),ui_1):
          if 'remotely' in k.lower() and 'AV' not in p.keys():
            p['AV']=k.strip().title()
          if 'UI' not in p.keys():
            p['UI']=k.strip().title()
          else:
            continue
        elif find_sub(k.lower(), ac) and 'AC' not in p.keys():
          p['AC']=k.strip().title()
        elif find_sub(k.lower(), av) and 'AV' not in p.keys():    
          p['AV']=k.strip().title()
          if find_sub(k.lower(), ui_2, 1):
            p['UI']=k.strip().title()
        elif find_sub(k.lower(), ui_3) and 'UI' not in p.keys():
          p['UI']=k.strip().title()
  # process metrics of one line format
  else:
    if '/' in line:
      phs = line.split('/')
    elif ';' in line:
      phs = line.split(';')
    else:
      phs = [line]
    # store phrases with metrics in the divided line
    for i in phs:
      if find_sub(i.lower(),ui_1):
        if find_sub(i.lower(), av) and 'AV' not in p.keys():
          p['AV']=i.strip().title()
        if 'UI' not in p.keys():
          p['UI']=i.strip().title()
        else:
          continue     
      elif find_sub(i.lower(), ac) and 'AC' not in p.keys():
        p['AC']=i.strip().title()
      elif find_sub(i.lower(), av) and 'AV' not in p.keys():
        p['AV']=i.strip().title()
        if find_sub(i.lower(), ui_2, 1):
          p['UI']=i.strip().title()
      elif find_sub(i.lower(), ui_3) and 'UI' not in p.keys():
        p['UI']=i.strip().title()
  return p

In [48]:
# A function to extract property and store 
def catch_prop(i):
  catch = 0
  AV_flag = 0
  UI_flag = 0
  
  prop = {}
  lines = read_file(i)
  # extract and store metric information
  for j in lines:
    j = re.sub('·','',j)
    sents = tokenize.sent_tokenize(j)
    # process metrics in one line
    for k in sents:
      m = re.search('attention:',k.lower())
      if m is not None:
        if not m.start():
          prop.update(properties(k[m.end():], 0))
          catch = 1
    # process metrics in a specific section
    if not catch:
      if AV_flag:      
        output = properties(tokenize.sent_tokenize(j),1)
        real_output = {}
        for key in output.keys():
          if key not in prop.keys() or key != 'AV':
            real_output[key] = output[key]
        prop.update(real_output)
        AV_flag = 0
      if UI_flag:
        output = properties(tokenize.sent_tokenize(j),1)
        real_output = {}
        for key in output.keys():
          if key not in prop.keys() or key != 'AV':
            real_output[key] = output[key]
        prop.update(real_output)
        UI_flag = 0
      if re.search('[A-Za-z]',j) is not None:
        s = re.search('[A-Za-z]',j).start()
        j = j[s:]
      if j.lower() == 'exploitability':
        AV_flag = 1
      if j.lower() == 'difficulty':
        UI_flag = 1
  
  return prop

### 3.2 Researcher Information


#### 3.2.1 Utilities

In [49]:
# Create a list of keywords to extract researcher information
# keywords for researchers
r_kwrds = ['(self-)?report(ed|s)?', 'discover(ed|s)?', 'coordinate(d|s)?', 
           'identif(y|ied)?', '(self-)?disclose(d|s)?', 'sent', 
           'notif(y|ied)?', 'found','(is|are) credited']
# keywords for researchers in passive tense           
pass_r_kwds = []
for i in range(len(r_kwrds)):
  pass_r_kwds.append('(was|were|is|are) ' + r_kwrds[i] + ' by?')

#### 3.2.2 Functions

In [50]:
# A function to find the first verb in a sentence
def find_first_verb(line):
  text = nlp(line)
  for i in text:
    if i.pos_ == 'VERB':
      return str(i)

In [51]:
# A function to extract researchers in a line
def div_researcher(line):

  div = len(line)    
  # process side case that begins with 'based'
  if line.split()[0].lower() == 'based':
    valid_sent = line.split(',')[-1]
    verb = find_first_verb(valid_sent)
    div = re.search(verb, valid_sent.lower()).start()
    return valid_sent[:div].strip()
  # process part of the sentence with researcher information in passive tense
  for k in pass_r_kwds:
    if re.search(k, line.lower()) is not None:
      div = re.search(k, line.lower()).end()
      res = line[div:]      
      word_pos = [0,0]
      text = nlp(res)
      # begin collecting information from words of certain POSes
      for t in text:
        if word_pos[0] in ['CCONJ','ADP']:
          conj = str(word_pos[1])
          if t.pos_ in ['VERB','DET']:
            div = re.search(conj, res.lower()).start()
            res = res[:div].strip()
            break   
        word_pos = [t.pos_, t]
      return res.strip()
  # process another form of researcher information
  for i in r_kwrds:
    if re.search(i, line.lower()) is not None:
      new_div = re.search(i, line.lower()).start()
      if new_div < div:
        div = new_div
  line = line[:div].rstrip('to').strip().strip(',')
  return line

### 3.3 Intrusions


#### 3.3.1 Utilities

In [52]:
# utilities for extracting intrusions
err = ['3.3 BACKGROUND','VULNERABILITY OVERVIEW','VULNERABILIY OVERVIEW']
gap = ['VULNERABILITY DETAILS','BACKGROUND','RESEARCHER','MITIGATION',
       'VUNLNERABILITY DETAILS','RESEARCHERS','EXPLOITABILITY']  
sec = ['VULNERABILITY OVERVIEW','VUNLNERABILITY OVERVIEW',
       'VULNERABILITY CHARACTERIZATION','VULNERABILITY',
       'FILE AND DIRECTORY INFORMATION EXPOSURE CWE-538']
regex_1 = '(result(s|ing)? in|causes?|allows?( attakers?)?|enables?| attackers? could|can) [A-Za-z ]+(,|.)'
regex_2 = '( attackers?.+(could|can) )'

#### 3.3.2 Functions

In [53]:
# A functin to pinpoint the sentence without CVE and not a divider as intrusion 
def if_only(lines):
  rest_of = []
  for i in lines:
    if find_sub(i,['CVE','CWE','CVSS','---------']):
      continue
    elif check_line(i):
      continue
    else:
      rest_of.append(i)
  # return the first sentence pinpointed
  if len(rest_of) == 1:
    return rest_of[0]

In [54]:
# A function to collect vulneralbilities from read lines
def collect_vul(file_name):
  attack_flag = 0
  act = 0
  sect_flag = 0
  line_flag = 0
  vul_detail = []

  for j in read_file(file_name):
    j = re.sub('·','',j)
    # skip if the following line of a divider is wrongly placed
    if act:
      act = 0
      if text_equal(j.upper(),err):
        continue
    # stop collecting when the end of the section occurs
    if (text_equal(j.upper(),gap) or j == 'Exploitability') and attack_flag:
      attack_flag = 0
    # collect vulneralbility related phrases
    if attack_flag: 
      s = find_key(j,['\['])
      line = j[:s]
      vul_detail = vul_detail + tokenize.sent_tokenize(line)
    # begin collecting after beginning dividers
    if text_equal(j.upper(),sec):
      attack_flag = 1
      act = 1
      sect_flag = 1
    
  return vul_detail

In [55]:
# A function to get intrusion
def get_intru(adv):
  # initiate intrusion as a void string
  intru=''
  init_flag = 0
  line_flag = 0
  lines = []
  # extract intrusion using regular expressions
  index = 0
  Vul = adv['Vulnerabilities']
  
  file_name = adv['Link'].split('/')[-1]+'.txt'
  
  for line in collect_vul(file_name):
    lines.append(line)
    # pinpoint intrusion using regular expression
    if line_flag:
      if re.search(regex_1, line.lower()) is not None:       
        intru = line
        line_flag = 0
      elif re.search(regex_2, line.lower()) is not None:
        intru = line
        line_flag = 0
    # select the only line not divider and without CVE info as intrusion
    if check_line(line):
      if intru == '':
        only = if_only(lines)
        if only is not None:
          intru = only
      line_flag = 1
      if init_flag:
        Vul[index]['Intrusion'] = intru
        intru = ''
        index += 1
      init_flag = 1
      lines = []
  # select the only line not divider and without CVE info as intrusion
  if init_flag:
    if intru == '':
      only = if_only(lines)
      if only is not None:
        intru = only
    Vul[index]['Intrusion'] = intru   

# 4 Run the processes

## 4.0 Run Text Reading

In [56]:
# Read all the files
files_in_range=[]
with open("valid_files.txt", "r") as f:
  files_in_range = f.readlines()
for i in range(len(files_in_range)):
  files_in_range[i] = files_in_range[i].rstrip()

In [57]:
# Read all the affected products sections
advs=[]
for fi in files_in_range:
  advs.append(collect_lines(fi))

## 4.1 Run Information Extraction

In [58]:
# Run the process to get advisories
advs = []
for f in files_in_range:
  adv = get_info(f)
  advs.append(adv)

In [59]:
# Give each advisory a name
for i in advs:
  file_name = i['Title']+'.txt'
  the_line = read_file(file_name)[1].strip()
  i['Title'] = ('('+i['Title']+') '+the_line)

## 4.2 Run Affected Products Extraction

In [120]:
# Run version conversion for all the advisories
for i in advs:
  file_name = i['Link'].split('/')[-1]+'.txt'
  i['Affected Products'] = ver_con(file_name)
  i['AP text'] = ', '.join(collect_lines(file_name)[1])

## 4.3 Run Metrics Extraction

In [60]:
# Run metric indentifying
for i in advs:
  i['AC'] = ''
  i['AV'] = ''
  i['UI'] = ''
  file_name = i['Link'].split('/')[-1]+'.txt'
  p = catch_prop(file_name)
  i.update(catch_prop(file_name))

In [61]:
# Convert metrics phrases into values
for i in advs:
  result = {'UI':{'text':i['UI'],'value':'None'}, 
            'AC':{'text':i['AC'],'value':''},
            'AV':{'text':i['AV'],'value':''}}
  expt = 0
  for key in met_val.keys():
    kwrds_met = met_val[key]
    if i[key] == '':
      continue
    for kwrd in kwrds_met.keys():
      if kwrd not in i[key].lower() and key == 'UI':
        result['UI']['value'] = 'Required'
      elif kwrd in i[key].lower():
        result[key]['value'] = kwrds_met[kwrd]
        if result[key]['value'] == 'case':
          if result['UI']['value'] == 'None':
            result[key]['value'] = 'High'
          elif result['UI']['value'] == 'Required':
            result[key]['value'] = 'Low'
        break
  i.update(result)

In [64]:
# Post process special cases of vulneralbility metrics 
for i in advs:
  av = i['AV']['text'].lower()
  for j in metric_sc:
    if re.search(j,av) is not None:
      if j == 'not ?.+(remotely| remotely)':
        if 'physical access' in j:
          i['AV']['value'] = 'Physical'
        else:
          i['AV']['value'] = 'Local'
      elif j == 'some':
        i['AV']['value'] = 'Network|Local'
      elif j == 'except for':
        i['AV']['value'] = 'Network|Local'
      elif j == '(one|three) of ':
        i['AV']['value'] = 'Network|Local'
      else:
        i['AV']['value'] = 'Network'
      break
  ac = i['AC']['text'].lower()
  if re.search('would not be difficult', ac) is not None:
    i['AC']['value'] = 'Low'
  ui = i['UI']['text'].lower()
  if re.search(' an anthenticated user ',ui) is not None:
    i['UI']['text'] = ''
    i['UI']['value'] = 'None'

In [63]:
# Post process special cases of vulneralbility metrics 
for adv in advs:
  i = adv['Link'].split('/')[-1]+'.txt'
  rf = read_file(i)
  num = len(rf)
  if not num:
    continue
  j = 0
  v = []
  while j < num:
    l = rf[j]
    if l.lower() == 'exploitability':
      temp = j+1
      nxt_one = rf[temp]
      if '---------' in nxt_one:
        temp = temp+1
        nxt_one = rf[temp]
      nxt = nxt_one
      while re.search('exisi?t(e|a)nce of (report|exp(loit|olit))',nxt.lower()) is None:
        if '---------' not in nxt:
          v.append(nxt)
        temp += 1
        nxt = rf[temp]
      break
    else:
      j += 1
  if len(v):
    flag = 0
    for t in typical:
      if re.search(t,v[0]) is None:
        flag += 1
      if flag == len(typical):
        text = ' '.join(v)
        adv['AV']['text'] = text
        adv['AV']['value'] = 'Network|Local'

## 4.4 Run Researcher and Intrusion Extraction

In [121]:
# correct extracted researcher information
for i in advs:
  if i['Researcher'].upper() == i['Researcher']:
    i['Researcher'] = ''
  elif '-----' in i['Researcher']:
    catch = 0
    file_name = i['Link'].split('/')[-1]+'.txt'
    for line in read_file(file_name):
      if catch:
        catch -= 1
        if not catch:
          i['Researcher'] = line
      if re.sub('[^A-Za-z]','',line.lower()) == 'researcher':
        catch = 2
# Run researcher extraction
for i in advs:
  R = i['Researcher']
  if R != '':
    i['Researcher text'] = R
    i['Researcher'] = div_researcher(R)
# Run intrusion extracting for advisories
for i in advs:
  get_intru(i)

# 5 Output


In [123]:
doc = {}
doc['Advisories'] = advs

In [124]:
with open('readable_file.json', 'w') as fp:
  json.dump(doc,fp)

In [125]:
with open('output_advisories.json', 'w') as fp:
  json.dump(doc, fp, indent = 4)